## Useful links
- vgg16 keras implementation - https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
- another vgg16 keras implementation - https://github.com/SamSamskies/courses/blob/master/deeplearning1/nbs/vgg16.py
- keras model API - https://keras.io/models/model/
- nvidia model implementation - https://github.com/jacobgil/keras-steering-angle-visualizations
- vehicle steering angle visualizations blog post - https://jacobgil.github.io/deeplearning/vehicle-steering-angle-visualizations